In [1]:
import subprocess
import pandas as pd

This script will run nl_01 - nl_03 scripts from start to finish.

01: All of HMDB

02: HMDB limited to endogenous molecules only.

03: HMDB limited to experimentally detected molecules only.

04: HMDB limited to spotted molecules only.

bits: Morgan 1024-bit chemical fingerprints.

mord_norm: Mordred chemical descriptors normalized 0-1.

fp_feasts: Morgan + Mordred

In [2]:
# Move all to same folder...
datasets = ['01_all_expt.pickle','02_en_expt.pickle',
            '03_endo_expt.pickle', '04_known_expt.pickle',
            '01_all_std_spots.pickle',
            '02_en_std_spots.pickle',
            '03_endo_std_spots.pickle',
            '04_known_std_spots.pickle'
           ]

In [3]:
data_data = {}
data_data['key'] = ('total', 'has_H2O', 'num_ds_id')
for d in datasets:
    x = pd.read_pickle(d)

    ids = x.shape[0]
    ds_count = len(x.ds_id.unique())
    trues = x.has_H2O.sum()
    data_data[d] = (ids, trues, ds_count)

In [4]:
data_data

{'key': ('total', 'has_H2O', 'num_ds_id'),
 '01_all_expt.pickle': (8387, 3743, 14),
 '02_en_expt.pickle': (7224, 3266, 14),
 '03_endo_expt.pickle': (3622, 1296, 14),
 '04_known_expt.pickle': (247, 78, 13),
 '01_all_std_spots.pickle': (3563, 1918, 4),
 '02_en_std_spots.pickle': (3240, 1829, 4),
 '03_endo_std_spots.pickle': (1817, 771, 4),
 '04_known_std_spots.pickle': (252, 80, 4)}

In [ ]:
final_out = {}
for d in datasets:
    cmd_1 = ('python nl_01_preprocess.py -f ' + d).split(" ")
    ret_val = subprocess.check_output(cmd_1, universal_newlines=True)
    print(ret_val)
    out_1a = ret_val.split('\n')[-3]
    out_1b = ret_val.split('\n')[-2]
    
    head = 'python nl_02_join.py --o '
    cmd_2 = head + out_1a + ' --h ' + out_1b
    cmd_2 = cmd_2.split(" ")
    ret_val = subprocess.check_output(cmd_2, universal_newlines=True)
    print(ret_val)
    out_2 = ret_val.split('\n')[-2]
    
    cmd_3 = ('python nl_03_filter_model.py --i ' + out_2).split(' ')
    ret_val = subprocess.check_output(cmd_3, universal_newlines=True)
    print(ret_val)
    out_3 = ret_val.split('\n')[-2]
        
    final_out[d] = out_3

In [60]:
final_out

{'01_all_expt.pickle': '01_all_expt_output_03.pickle'}

In [54]:

z = pd.read_pickle('01_all_expt_output_03.pickle')

In [55]:
z

,model,polarity,struct_target,obs_loss_type,prediction,any_fdr,any_coloc,n_train_test_total,n_train_test_true,accuracy_train,accuracy_test,specificity,sensitivity,f1
1,direct,negative,H2Oe,n_loss_wparent,H2O_Present,0.20,0.00,28,6,0.500000,0.500000,1.0,0.363636,0.461538
1,direct,negative,H2Oe,n_loss_wparent,H2O_Present,0.20,0.00,28,6,0.500000,0.500000,1.0,0.363636,0.461538
2,direct,negative,H2Oe,n_loss_wparent,H2O_Present,0.20,0.50,24,2,0.416667,0.416667,1.0,0.363636,0.222222
3,direct,negative,H2Oe,n_loss_wparent,H2O_Present,0.20,0.75,24,2,0.416667,0.416667,1.0,0.363636,0.222222
4,direct,negative,H2Oe,n_loss_wparent,H2O_Present,0.10,0.00,12,4,0.666667,0.666667,1.0,0.500000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,random_forest,positive,H2Oe,n_loss_wparent,fp_feats,0.10,0.50,"(4, 2, 6)","(2, 0, 2)",1.000000,1.000000,NaN,NaN,NaN
141,random_forest,positive,H2Oe,n_loss_wparent,fp_feats,0.10,0.75,"(4, 2, 6)","(2, 0, 2)",1.000000,1.000000,NaN,NaN,NaN
142,random_forest,positive,H2Oe,n_loss_wparent,fp_feats,0.05,0.00,"(1, 1, 2)","(1, 1, 2)",1.000000,1.000000,NaN,NaN,NaN
143,random_forest,positive,H2Oe,n_loss_wparent,fp_feats,0.05,0.50,"(1, 1, 2)","(1, 1, 2)",1.000000,1.000000,NaN,NaN,NaN


In [48]:
z.to_csv('test.csv')